---
# CitiBike Notebook
---

In [1]:
import numpy as np
import pandas as pd
import json
import urllib.request
import veroviz as vrv
import os

--- 
## 1. Import Station Info (from .json)
---

In [2]:
# Grab JSON data:
with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_information.json") as url:
    station_info_data = json.loads(url.read().decode())
station_info_data

{'last_updated': 1582578030,
 'ttl': 10,
 'data': {'stations': [{'station_id': '304',
    'external_id': '66db6da2-0aca-11e7-82f6-3863bb44ef7c',
    'name': 'Broadway & Battery Pl',
    'short_name': '4962.01',
    'lat': 40.70463334,
    'lon': -74.01361706,
    'region_id': 71,
    'rental_methods': ['CREDITCARD', 'KEY'],
    'capacity': 33,
    'rental_url': 'http://app.citibikenyc.com/S6Lr/IBV092JufD?station_id=304',
    'electric_bike_surcharge_waiver': False,
    'eightd_has_key_dispenser': True,
    'eightd_station_services': [{'id': 'a58d9e34-2f28-40eb-b4a6-c8c01375657a',
      'service_type': 'ATTENDED_SERVICE',
      'bikes_availability': 'UNLIMITED',
      'docks_availability': 'NONE',
      'name': 'Valet Service',
      'description': 'Citi Bike Valet Attendant Service Available',
      'schedule_description': '',
      'link_for_more_info': 'https://www.citibikenyc.com/valet'}],
    'has_kiosk': True},
   {'station_id': '359',
    'external_id': '66dbc982-0aca-11e7-82f6-3

In [3]:
# Convert the JSON data into a Pandas dataframe:
station_info_df = pd.DataFrame(station_info_data['data']['stations'])
station_info_df.head()

,station_id,external_id,name,short_name,lat,lon,region_id,rental_methods,capacity,rental_url,electric_bike_surcharge_waiver,eightd_has_key_dispenser,eightd_station_services,has_kiosk
0,304,66db6da2-0aca-11e7-82f6-3863bb44ef7c,Broadway & Battery Pl,4962.01,40.704633,-74.013617,71,"[CREDITCARD, KEY]",33,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,True,[{'id': 'a58d9e34-2f28-40eb-b4a6-c8c01375657a'...,True
1,359,66dbc982-0aca-11e7-82f6-3863bb44ef7c,E 47 St & Park Ave,6584.12,40.755103,-73.974987,71,"[CREDITCARD, KEY]",64,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,[{'id': '2e104e31-606a-44af-8b25-ceaffc338489'...,True
2,367,66dbcdfc-0aca-11e7-82f6-3863bb44ef7c,E 53 St & Lexington Ave,6617.09,40.758281,-73.970694,71,"[CREDITCARD, KEY]",34,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,[{'id': '2d9a5c9e-50e0-4aed-a63b-91ca81e7d2c0'...,True
3,402,66dbf0d0-0aca-11e7-82f6-3863bb44ef7c,Broadway & E 22 St,6098.07,40.740343,-73.989551,71,"[CREDITCARD, KEY]",39,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,[{'id': '37a1ae1b-3dd6-4876-8c57-572aaac97981'...,True
4,3255,66ddbd20-0aca-11e7-82f6-3863bb44ef7c,8 Ave & W 31 St,6450.05,40.750585,-73.994685,71,"[CREDITCARD, KEY]",19,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,[{'id': '9fb74cf0-b08b-4983-ae0e-be909fc28bc3'...,True


---
## 2.  Import Station Status (from .json)
---

In [4]:
# Grab JSON data:
with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
    station_status_data = json.loads(url.read().decode())
station_status_data

{'last_updated': 1582578030,
 'ttl': 10,
 'data': {'stations': [{'station_id': '304',
    'num_bikes_available': 13,
    'num_ebikes_available': 0,
    'num_bikes_disabled': 4,
    'num_docks_available': 16,
    'num_docks_disabled': 0,
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 0,
    'last_reported': 1582577834,
    'eightd_has_available_keys': True,
    'eightd_active_station_services': [{'id': 'a58d9e34-2f28-40eb-b4a6-c8c01375657a'}]},
   {'station_id': '359',
    'num_bikes_available': 28,
    'num_ebikes_available': 0,
    'num_bikes_disabled': 1,
    'num_docks_available': 35,
    'num_docks_disabled': 0,
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'last_reported': 1582577907,
    'eightd_has_available_keys': False,
    'eightd_active_station_services': [{'id': '2e104e31-606a-44af-8b25-ceaffc338489'}]},
   {'station_id': '367',
    'num_bikes_available': 11,
    'num_ebikes_available': 0,
    'num_bikes_disabled': 0,
    'num_docks

In [5]:
# Convert the data into a Pandas dataframe, AND add "color" columns to be used for nodes:
station_status_df = pd.DataFrame(station_status_data['data']['stations'])
station_status_df['colors'] = None
station_status_df['CESIUMcolors'] = None

In [6]:
# Populate "color" columns depending on the available docks and bikes at a station
pd.set_option('mode.chained_assignment', None)
for i in station_status_df.index:
    if station_status_df['num_docks_available'][i] >= 6 and station_status_df['num_bikes_available'][i] >= 6:
        station_status_df['colors'][i] = 'green'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.GREEN'
    elif station_status_df['num_docks_available'][i] <= 5:
        station_status_df['colors'][i] = 'lightred'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.YELLOW'
    elif station_status_df['num_bikes_available'][i] <= 5:
        station_status_df['colors'][i] = 'red'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.RED'
station_status_df.head()

,station_id,num_bikes_available,num_ebikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,eightd_has_available_keys,eightd_active_station_services,colors,CESIUMcolors
0,304,13,0,4,16,0,1,1,0,1582577834,True,[{'id': 'a58d9e34-2f28-40eb-b4a6-c8c01375657a'}],green,Cesium.Color.GREEN
1,359,28,0,1,35,0,1,1,1,1582577907,False,[{'id': '2e104e31-606a-44af-8b25-ceaffc338489'}],green,Cesium.Color.GREEN
2,367,11,0,0,23,0,1,1,1,1582577619,False,[{'id': '2d9a5c9e-50e0-4aed-a63b-91ca81e7d2c0'}],green,Cesium.Color.GREEN
3,402,9,0,2,28,0,1,1,1,1582577796,False,[{'id': '37a1ae1b-3dd6-4876-8c57-572aaac97981'}],green,Cesium.Color.GREEN
4,3255,4,0,4,11,0,1,1,1,1582578018,False,[{'id': '9fb74cf0-b08b-4983-ae0e-be909fc28bc3'}],red,Cesium.Color.RED


--- 
## 3.  Import Trip Data (from .csv)
---

In [7]:
# Trip data
bike_trips_df = pd.read_csv('citibike_tripdata.csv')
bike_trips_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1


--- 
## 4. Create a VeRoViz "nodes" Dataframe
---

In [8]:
# Nodes DF
nodes = vrv.initDataframe('nodes')
nodes['id'] = station_info_df['station_id'].values
nodes[['id', 'lat', 'lon', 'nodeName']] = station_info_df[['station_id', 'lat', 'lon', 'name']].values
nodes[['leafletIconText', 'cesiumIconText']] = station_info_df[['name', 'station_id']].values
nodes['leafletColor'] = station_status_df['colors'].values
nodes['cesiumColor'] = station_status_df['CESIUMcolors'].values
nodes.loc[:,'altMeters'] = 0
nodes.loc[:,['nodeType', 'leafletIconPrefix', 'leafletIconType']] = [
             'Citibike Station', 'fa', 'bicycle']
nodes.loc[:,'cesiumIconType'] = 'pin'
nodes.head()

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,304,40.7046,-74.0136,0,Broadway & Battery Pl,Citibike Station,fa,bicycle,green,Broadway & Battery Pl,pin,Cesium.Color.GREEN,304
1,359,40.7551,-73.975,0,E 47 St & Park Ave,Citibike Station,fa,bicycle,green,E 47 St & Park Ave,pin,Cesium.Color.GREEN,359
2,367,40.7583,-73.9707,0,E 53 St & Lexington Ave,Citibike Station,fa,bicycle,green,E 53 St & Lexington Ave,pin,Cesium.Color.GREEN,367
3,402,40.7403,-73.9896,0,Broadway & E 22 St,Citibike Station,fa,bicycle,green,Broadway & E 22 St,pin,Cesium.Color.GREEN,402
4,3255,40.7506,-73.9947,0,8 Ave & W 31 St,Citibike Station,fa,bicycle,red,8 Ave & W 31 St,pin,Cesium.Color.RED,3255


In [9]:
# This block preprocesses the data to get the station attributes for one specicific bike

# Pull out one bike to investigate
bike_trips_df2 = bike_trips_df[bike_trips_df['bikeid'] == 14530]

# Extract station name, station id, latitude, and longitude
_stations_name = bike_trips_df2['start station name'].tolist()
_stations_name.extend(bike_trips_df2['end station name'].tolist())
_stations_lat = bike_trips_df2['start station latitude'].tolist()
_stations_lat.extend(bike_trips_df2['end station latitude'].tolist())
_stations_lon = bike_trips_df2['start station longitude'].tolist()
_stations_lon.extend(bike_trips_df2['end station longitude'].tolist())
_stations_id = bike_trips_df2['start station id'].tolist()
_stations_id.extend(bike_trips_df2['end station id'].tolist())

# Create a DF with the above attributes. This will be used to populate a nodes DF for the chosen bike
_stations_df = pd.DataFrame(columns = ['station_id', 'lat', 'lon', 'name'])
_stations_df['station_id'] = _stations_id
_stations_df['lat'] = _stations_lat
_stations_df['lon'] = _stations_lon
_stations_df['name'] = _stations_name
_stations_df = _stations_df.drop_duplicates(subset='station_id')

# Merge this DF with the 'station_status' DF in order to get the proper colors for the nodes
# that the chosen bike visited
_stations_df['station_id']=_stations_df['station_id'].astype(int)
station_status_df['station_id']=station_status_df['station_id'].astype(int)
_stations_df = _stations_df.merge(station_status_df, left_on = 'station_id', right_on = 'station_id')
_stations_df.head()

,station_id,lat,lon,name,num_bikes_available,num_ebikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,eightd_has_available_keys,eightd_active_station_services,colors,CESIUMcolors
0,261,40.694749,-73.983625,Johnson St & Gold St,9,0,0,18,0,1,1,1,1582576428,False,NaN,green,Cesium.Color.GREEN
1,2000,40.702551,-73.989402,Front St & Washington St,29,0,1,0,0,1,1,1,1582577552,False,NaN,lightred,Cesium.Color.YELLOW
2,3414,40.680945,-73.975673,Bergen St & Flatbush Ave,24,0,0,9,0,1,1,1,1582578002,False,NaN,green,Cesium.Color.GREEN
3,3486,40.688417,-73.984517,Schermerhorn St & Bond St,30,0,0,0,0,1,1,1,1582575874,False,NaN,lightred,Cesium.Color.YELLOW
4,241,40.689810,-73.974931,DeKalb Ave & S Portland Ave,3,0,0,20,0,1,1,1,1582576601,False,NaN,red,Cesium.Color.RED


In [10]:
# New nodes DF -- only for bikeid 14530
nodes2 = vrv.initDataframe('nodes')

#Populate nodes DF with '_stations_df'
nodes2['id'] = _stations_df['station_id'].values
nodes2[['id', 'lat', 'lon', 'nodeName']] = _stations_df[['station_id', 'lat', 'lon', 'name']].values
nodes2[['leafletIconText', 'cesiumIconText']] = _stations_df[['name', 'station_id']].values
nodes2['leafletColor'] = _stations_df['colors'].values
nodes2['cesiumColor'] = _stations_df['CESIUMcolors'].values
nodes2.loc[:,'altMeters'] = 0
nodes2.loc[:,['nodeType', 'leafletIconPrefix', 'leafletIconType']] = [
             'Citibike Station', 'fa', 'bicycle']
nodes2.loc[:,'cesiumIconType'] = 'pin'
nodes2.head()

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,261,40.694749,-73.983625,0,Johnson St & Gold St,Citibike Station,fa,bicycle,green,Johnson St & Gold St,pin,Cesium.Color.GREEN,261
1,2000,40.702551,-73.989402,0,Front St & Washington St,Citibike Station,fa,bicycle,lightred,Front St & Washington St,pin,Cesium.Color.YELLOW,2000
2,3414,40.680945,-73.975673,0,Bergen St & Flatbush Ave,Citibike Station,fa,bicycle,green,Bergen St & Flatbush Ave,pin,Cesium.Color.GREEN,3414
3,3486,40.688417,-73.984517,0,Schermerhorn St & Bond St,Citibike Station,fa,bicycle,lightred,Schermerhorn St & Bond St,pin,Cesium.Color.YELLOW,3486
4,241,40.689810,-73.974931,0,DeKalb Ave & S Portland Ave,Citibike Station,fa,bicycle,red,DeKalb Ave & S Portland Ave,pin,Cesium.Color.RED,241


In [11]:
# Show all of the nodes on a Leaflet map:
vrv.createLeaflet(nodes=nodes)

In [12]:
# Show nodes only for bike 14530 on Leaflet map:
vrv.createLeaflet(nodes=nodes2)

In [116]:
# DONE
# - green  --> bikes and docks are available
# - red    --> no bikes available
# - yellow (lightred for Leaflet) --> no docks available

--- 
## 5. Create a VeRoViz "assignments" Dataframe
---

In [13]:
arcs = vrv.initDataframe('arcs')
list(arcs.columns)

['odID',
 'objectID',
 'startLat',
 'startLon',
 'endLat',
 'endLon',
 'leafletColor',
 'leafletWeight',
 'leafletStyle',
 'leafletOpacity',
 'useArrows',
 'cesiumColor',
 'cesiumWeight',
 'cesiumStyle',
 'cesiumOpacity']

In [14]:
# What is the first start time in our bike_trips_df?
min(bike_trips_df['starttime'])

'2020-01-01 00:00:55.3900'

In [15]:
# This next command will produce a "timestamp" (days HH:MM:SS.ms) 
# showing the time since the first observed `starttime`:
bike_trips_df['timeAfterStart'] = pd.to_datetime(bike_trips_df['starttime']) - \
                                  pd.to_datetime(min(bike_trips_df['starttime']))

# Now, convert this to a decimal number of seconds:
bike_trips_df['timeAfterStart'] = bike_trips_df['timeAfterStart'].dt.total_seconds().astype(int)
bike_trips_df['timeAfterStart'].head()

0     0
1    12
2    46
3    50
4    50
Name: timeAfterStart, dtype: int64

In [16]:
# Just for fun, here's the time differences between start/stop times:
pd.to_datetime(bike_trips_df['stoptime']) - pd.to_datetime(bike_trips_df['starttime'])

0         00:13:09.757000
1         00:25:41.076000
2         00:24:24.871000
3         00:09:52.594000
4         00:11:42.452000
                ...      
1240591   00:26:27.607000
1240592   00:03:42.831000
1240593   00:02:43.862000
1240594   00:05:27.148000
1240595   00:08:04.146000
Length: 1240596, dtype: timedelta64[ns]

In [17]:
# Assignments DF:
assignments = vrv.initDataframe('assignments')

# Copy over the static values.
# Start by copying a single column to avoid the size mis-match issue:
assignments['objectID'] = bike_trips_df['bikeid'].values
assignments[['startLat', 'startLon', 'endLat', 'endLon']] = bike_trips_df[['start station latitude', 
                                                                          'start station longitude',
                                                                          'end station latitude',
                                                                          'end station longitude']].values

# Copy our new calculated column:
assignments['startTimeSec'] = bike_trips_df['timeAfterStart'].values

# Use the calculated column and tripduration to get the end time (in seconds):
assignments['endTimeSec'] = (bike_trips_df['timeAfterStart'] + bike_trips_df['tripduration']).values

# Fill in the rest of our assignments df with some hard-coded values:
assignments.loc[:,['modelFile', 'modelScale', 'modelMinPxSize', 'startAltMeters', 'endAltMeters', 
                   'leafletColor', 'leafletWeight', 'leafletStyle', 'leafletOpacity', 'useArrows',
                   'cesiumColor', 'cesiumWeight', 'cesiumStyle', 'cesiumOpacity']] = \
                  ['veroviz/models/car_green.gltf', 100, 45, 0, 0, 
                   'blue', 2, 'solid', 0.8, False, 
                   'Cesium.Color.BLUE', 2, 'solid', 0.7]

# Finally (for now), let's generate a unique odID value for each row.
# This will make sense only if we assume that each row corresponds to a specific
# O/D pair.  Conversely, if we have turn-by-turn arcs, we'll need to group
# multiple rows into the same O/D pair.  We'll tackle that case if/when 
# we encounter it.
assignments['odID'] = bike_trips_df.index
assignments

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,30326,veroviz/models/car_green.gltf,100,45,0,40.732219,-73.981656,0,789,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1,1,17105,veroviz/models/car_green.gltf,100,45,12,40.661063,-73.979453,0,1553,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
2,2,40177,veroviz/models/car_green.gltf,100,45,46,40.743227,-73.974498,0,1510,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
3,3,27690,veroviz/models/car_green.gltf,100,45,50,40.736529,-74.006180,0,642,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
4,4,32583,veroviz/models/car_green.gltf,100,45,50,40.694546,-73.958014,0,752,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240591,1240591,40662,veroviz/models/car_green.gltf,100,45,2678311,40.731437,-73.994903,0,2679898,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1240592,1240592,28722,veroviz/models/car_green.gltf,100,45,2678317,40.735238,-74.000271,0,2678539,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1240593,1240593,32530,veroviz/models/car_green.gltf,100,45,2678323,40.720874,-73.980858,0,2678486,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1240594,1240594,15314,veroviz/models/car_green.gltf,100,45,2678333,40.732233,-73.988900,0,2678660,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7


In [18]:
# Pull out the chosen bike:
assignments2 = assignments[assignments['objectID'] == 14530]
assignments2 = assignments2.reset_index(drop = True)
assignments2

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,26822,14530,veroviz/models/car_green.gltf,100,45,120256,40.694749,-73.983625,0,120804,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1,50614,14530,veroviz/models/car_green.gltf,100,45,152149,40.702551,-73.989402,0,153210,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
2,134556,14530,veroviz/models/car_green.gltf,100,45,397566,40.680945,-73.975673,0,397966,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
3,171702,14530,veroviz/models/car_green.gltf,100,45,490146,40.688417,-73.984517,0,490447,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
4,175456,14530,veroviz/models/car_green.gltf,100,45,494095,40.689810,-73.974931,0,494262,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,1183146,14530,veroviz/models/car_green.gltf,100,45,2572193,40.751726,-73.987535,0,2573882,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
70,1186951,14530,veroviz/models/car_green.gltf,100,45,2575799,40.722104,-73.997249,0,2576187,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
71,1191584,14530,veroviz/models/car_green.gltf,100,45,2584280,40.717227,-73.988021,0,2585164,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
72,1200708,14530,veroviz/models/car_green.gltf,100,45,2622419,40.734232,-73.986923,0,2622934,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7


In [19]:
# Create a new DF containing the repositioning of the chosen bike
reposition_df = pd.DataFrame(columns = ['objectID', 'endLat', 'endLon', 'endTimeSec',\
                                        'startLat', 'startLon', 'startTimeSec'])
reposition_endLat = []
reposition_endLon = []
reposition_startLat = []
reposition_startLon = []
reposition_endtime = []
reposition_starttime = []
for i in range(len(assignments2)-1):
    if assignments2['endLat'][i] != assignments2['startLat'][i+1]:
        reposition_endLat.append(assignments2['endLat'][i])
        reposition_endLon.append(assignments2['endLon'][i])
        reposition_endtime.append(assignments2['endTimeSec'][i])
        reposition_startLat.append(assignments2['startLat'][i+1])
        reposition_startLon.append(assignments2['startLon'][i+1])
        reposition_starttime.append(assignments2['startTimeSec'][i+1])
    else:
        pass
reposition_df['endLat'] = reposition_endLat
reposition_df['endLon'] = reposition_endLon
reposition_df['endTimeSec'] = reposition_endtime
reposition_df['startLat'] = reposition_startLat
reposition_df['startLon'] = reposition_startLon
reposition_df['startTimeSec'] = reposition_starttime
reposition_df['objectID'] = 14530
reposition_df

,objectID,endLat,endLon,endTimeSec,startLat,startLon,startTimeSec
0,14530,40.689622,-73.983043,511122,40.678612,-73.990373,566536
1,14530,40.733320,-73.995101,1331282,40.729538,-73.984267,1339185
2,14530,40.734814,-73.992085,1339473,40.732219,-73.981656,1415672
3,14530,40.745497,-74.001971,1444377,40.766741,-73.979069,1608598


In [20]:
# New assignment DF for repositioning
assignments3 = vrv.initDataframe('assignments')

# Copy over the static values.
# Start by copying a single column to avoid the size mis-match issue:
assignments3['objectID'] = reposition_df['objectID'].values
assignments3[['startLat', 'startLon', 'endLat', 'endLon']] = reposition_df[['endLat', 
                                                                          'endLon',
                                                                          'startLat',
                                                                          'startLon']].values

# Copy our new calculated column:
assignments3['startTimeSec'] = reposition_df['startTimeSec'].values

# Use the calculated column and tripduration to get the end time (in seconds):
assignments3['endTimeSec'] = reposition_df['endTimeSec'].values

# Fill in the rest of our assignments df with some hard-coded values:
assignments3.loc[:,['modelFile', 'modelScale', 'modelMinPxSize', 'startAltMeters', 'endAltMeters', 
                   'leafletColor', 'leafletWeight', 'leafletStyle', 'leafletOpacity', 'useArrows',
                   'cesiumColor', 'cesiumWeight', 'cesiumStyle', 'cesiumOpacity']] = \
                  ['veroviz/models/car_green.gltf', 100, 45, 0, 0, 
                   'red', 2, 'solid', 0.8, False, 
                   'Cesium.Color.RED', 2, 'solid', 0.7]

# Finally (for now), let's generate a unique odID value for each row.
# This will make sense only if we assume that each row corresponds to a specific
# O/D pair.  Conversely, if we have turn-by-turn arcs, we'll need to group
# multiple rows into the same O/D pair.  We'll tackle that case if/when 
# we encounter it.
assignments3.loc[:,'odID'] = list(range(0, len(assignments3)))
assignments3

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,0,14530,veroviz/models/car_green.gltf,100,45,566536,40.689622,-73.983043,0,511122,...,0,red,2,solid,0.8,False,Cesium.Color.RED,2,solid,0.7
1,1,14530,veroviz/models/car_green.gltf,100,45,1339185,40.733320,-73.995101,0,1331282,...,0,red,2,solid,0.8,False,Cesium.Color.RED,2,solid,0.7
2,2,14530,veroviz/models/car_green.gltf,100,45,1415672,40.734814,-73.992085,0,1339473,...,0,red,2,solid,0.8,False,Cesium.Color.RED,2,solid,0.7
3,3,14530,veroviz/models/car_green.gltf,100,45,1608598,40.745497,-74.001971,0,1444377,...,0,red,2,solid,0.8,False,Cesium.Color.RED,2,solid,0.7


--- 
## 6. Create a Leaflet map 
---

In [21]:
# Show all of the ride arcs for the chosen bike:
vrv.createLeaflet(arcs=assignments2)

In [22]:
# Show all of the repositioning arcs for the chosen bike:
vrv.createLeaflet(arcs=assignments3)

In [191]:
# DONE

# Some (most?) of our bikes are being re-positioned by the CitiBike staff.
# We'll want to identify when this happens.

# One way to identify that a re-positioning has occurred is if 
# The start location for the next arc of a given bike does not 
# match the end location of the previous arc for this bike.

# We might also want to know when these activities occur.
# Is it only at night?  Do we have enough data to figure this out?

# OPTION 1: Use a "for" loop <--
# OPTION 2: Don't use a "for" loop
# Which is faster?
# ---> timeit()
#      import timeit
#      %timeit
# ---> current time - previous time

# Draw on a (separate) Leaflet map and a (separate?) Cesium movie the repositioning of bikes
# Use vrv.addStaticAssignment() to display stationary bikes.
#     ?Can we just append to the end of assignments dataframe
#     ?If not, is there a Pandas "insert" function (to automatically re-index a df)

--- 
## 7. Create a Cesium movie for one bike
---

In [23]:
# startDate: Format is "YYYY-MM-DD"
startDate = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%Y-%m-%d')

# startTime: Format is "HH:MM:SS"
startTime = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%H:%M:%S')

vrv.createCesium(
    assignments = assignments2,
    nodes       = nodes2,
    startDate   = startDate,
    startTime   = startTime,
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'IE_670/citibike_trips')

Message: File selector was written to /Users/kylehunt/development/Cesium/IE_670/citibike_trips/;IE_670;citibike_trips.vrv ...
Message: Configs were written to /Users/kylehunt/development/Cesium/IE_670/citibike_trips/config.js ...
Message: Nodes were written to /Users/kylehunt/development/Cesium/IE_670/citibike_trips/displayNodes.js ...
Message: Assignments (.js) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_trips/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_trips/routes.czml ...


In [24]:
# startDate: Format is "YYYY-MM-DD"
startDate = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%Y-%m-%d')

# startTime: Format is "HH:MM:SS"
startTime = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%H:%M:%S')

vrv.createCesium(
    assignments = assignments3,
    nodes       = nodes2,
    startDate   = startDate,
    startTime   = startTime,
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'IE_670/citibike_repositioning')

Message: File selector was written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/;IE_670;citibike_repositioning.vrv ...
Message: Configs were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/config.js ...
Message: Nodes were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/displayNodes.js ...
Message: Assignments (.js) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/routes.czml ...


In [151]:
# DONE

# The Cesium movie is cluttered with all of our station markers.
# It would be better to only include the markers that are actually relevant 
# to our given bike.

# Fortunately, our bike trips df contains the station IDs.
# We just need to get a list of unique IDs, and then 
# pass to createCesium only the subset of nodes corresponding to these IDs.

--- 

#### Playing around with dates/times
- Here's some code related to formatting dates/times.  There might be something useful here in the future...

In [181]:
pd.to_datetime(bike_trips_df['starttime']).dt.date

0          2020-01-01
1          2020-01-01
2          2020-01-01
3          2020-01-01
4          2020-01-01
              ...    
1240591    2020-01-31
1240592    2020-01-31
1240593    2020-01-31
1240594    2020-01-31
1240595    2020-01-31
Name: starttime, Length: 1240596, dtype: object

In [177]:
pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%Y-%m-%d')

'2020-01-01'

In [179]:
pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%H:%M:%S')

'00:00:55'